<a href="https://colab.research.google.com/github/hwan95/HonegongML/blob/main/CH5_2_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#그리드 서치(Grid Search)
: 하이퍼파라미터 탐색과 교차검증을 한번에 수행
```
from sklearn.model_selection import GridSearchCV
params={'param_name':[searching values list]}
gs = GridSearchCV(DecisionClassifier(random_state = 42),params, n_jobs=-1) 
```
**키워드**
- 검증세트(validation set)
- 교차 검증(cross validation)


## 교차 검증(cross validation)

: 검증 세트를 떼어내어 평가하는 과정을 여러번 반복한다

**검증 세트(validation set)** 는 테스트 세트를 사용하지 않고 모델을 평가하기 위한 것으로 훈련세트의 일부분을 사용한다.

**k_fold cross validation** 훈련세트를 k 부분으로 나누어 (k-1)개 세트를 훈련세트로, 1개 세트를 검증 세트로 나누어 모델을 훈련 및 평가하는 과정을 k번 반복하는 교차 검증 방법

```
from sklearn.model_selection import cross validate
cross_validate(dt, train_input, target_input) # default = 5-fold
```

##검증 세트 구현

In [ ]:
# 데이터 불러오기
import pandas as pd
import numpy as np
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
#배열 저장
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
#훈련세트와 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42
)
# 검증세트(val) 만들기
sub_input, val_input, sub_target, val_target =train_test_split(
    train_input, train_target, test_size =0.2, random_state =42
)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
# 모델 생성 및 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
# 훈련세트, 검증세트 점수 확인
print(dt.score(sub_input, sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


##교차검증 구현


- cross_validate() : test_score 값만 변환
-StratifiedKFold() : 타킷 클래스를 골고루 분할
-분할기 splitter 지정: 교차검증 할때 훈련세트 섞기 

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input, train_target)
print(scores)

{'fit_time': array([0.0096755 , 0.01633191, 0.01693153, 0.01321864, 0.00906491]), 'score_time': array([0.0012691 , 0.00125146, 0.0013032 , 0.00133276, 0.00085235]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
# 검증 테스트 점수 평균
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
#타킷 클래스 골고루 분할
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,train_input, train_target, cv= StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
# 훈련세트를 섞은후 10-폴드 교차검증
splitter = StratifiedKFold(n_splits = 10, shuffle=True, random_state =42)
score =cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.855300214703487


## 그리드 서치(Grid search)

: 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듭니다
```
n_jobs : 실행에 사용할 CPU코어수, default =1
n_jobs=-1 : 모든 cpu코어 사용

gs.best_estimator_ : 검증 전수가 가장 높은 조합의 모델

gs.best_params_: 그리드서치로 찾은 최적의 매개변수들

gs.cv_results_['mean_test_score']: 교차검증 평균점수

np.argmax(): 가장 큰값의 인덱스 추출
```

In [ ]:
#min_impurty_decrease 의 최적값 찾기
from sklearn.model_selection import GridSearchCV
params ={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state =42),params, n_jobs =-1)
#n_jobs : 실행에 사용할 CPU코어수, default =1

In [ ]:
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
# 검증 점수가 가장높은 모델
dt= gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
#그리드 서치로 찾은 최적의 매개변수
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
#교차검증 평균점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
#가장 큰 점수 출력
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
# 트리 깊이 제한 및 노드의 최소샘플수 지정

params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)   
}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


##랜덤 서치(Radom Search) 
: 매개변수를 샘플링할  수 있는 확률 분표를 전달

매개 변수의 값의 범위나 간격을 정하기 어려운 경우, 변수조건이 까다로운 경우, 연속적인 실수값인 경우에 사용한다. 

```
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state =42), params, n_iter = 100, n_jobs =-1, random_state=42)

```

In [ ]:
from scipy.stats import uniform, randint
rgen = randint(0,10)
rgen.rvs(10)

array([8, 5, 6, 3, 0, 1, 7, 9, 8, 3])

In [ ]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105, 100, 115, 101,  86, 100,  98,  89, 109,  97]))

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.29734983, 0.7962443 , 0.81693173, 0.23058552, 0.706598  ,
       0.67881209, 0.6621744 , 0.03608869, 0.52871646, 0.74099364])

In [ ]:
#탐색할 매개변수의 딕셔너리 생성
params ={'min_impurity_decrease': uniform(0.0001,0.001),
         'max_depth': randint(20,50),
         'min_samples_split':randint(2,25),
         'min_samples_leaf': randint(1,25,) 
    
}
#min_samples_leaf : 리프노드의 최소샘플 수

In [ ]:
#랜덤 서치 실행
from sklearn.model_selection import RandomizedSearchCV
gs= RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100
                       ,n_jobs = -1, random_state =42)
gs.fit(train_input,train_target)
#매개변수 조합 확인
print(gs.best_params_)
#교자 검증 최고점 확인
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [ ]:
#최종모델 결정 및 테스트 세트확인
dt= gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


### 예제
```
DecisionClassifier(splitter = 'random')
```
splitter='random' ( default = 'best' ) 이면 각 노드를 무작위로 분할한 후 가장 좋은 것을 고른다.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs= RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'), params, n_iter=100
                       ,n_jobs = -1, random_state =42)
gs.fit(train_input,train_target)
#매개변수 조합 확인
print(gs.best_params_)
#교자 검증 최고점 확인
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981


In [ ]:
dt= gs.best_estimator_
print(dt.score(test_input, test_target))

0.786923076923077
